In [ ]:
import sys
import numpy as np
import copy
# import os
# import requests
import random

In [ ]:
vertex_list = []
vertex_dict = {}

with open("_f370cd8b4d3482c940e4a57f489a200b_kargerMinCut.txt") as file:
    for line in file.readlines():
        a = list(map(int, line.split()))
        vertex_list.append(a[0])
        vertex_dict[a[0]] = a[1:]

## each element is a tuple of (vertex, vertex)
edge_list = [(x,y) for x in vertex_dict for y in vertex_dict[x]]
len(edge_list)

In [ ]:
vertex_dict_copy = copy.deepcopy(vertex_dict)

def edge_contraction(v1, v2, vertex_dict_arg):
    """
    Input:  2 vertices -- integer.
    Output: the edges of the contracted vertex -- list of integers.
    """
    edges = [j for id in [v1,v2]
               for j in vertex_dict_arg[id]
               if j not in [v1, v2]]
    return edges

def update_graph(v1, v2, vertex_dict_arg):
    
    ## Keep the vertext with the smaller integer
    to_drop = max(v1, v2)
    to_remain = min(v1, v2)

    ## associate the new vertex with the edges we created 
    contracted_edge = edge_contraction(to_drop,to_remain, vertex_dict_arg)
    vertex_dict_arg.pop(to_drop, None)
    vertex_dict_arg.pop(to_remain, None)
    
    ## update all the edges in the other vertices
    ## replace relevant vertices with the new vertex.
    for key in vertex_dict_arg.keys():
        if to_drop in vertex_dict_arg[key]:
            vertex_dict_arg[key] = \
            [to_remain if x==to_drop else x for x in vertex_dict_arg[key]]
    
    vertex_dict_arg[to_remain] = contracted_edge
    
    ## update vertex_vertex_tuple
    contracted_edge_list = [(x,y) for x in vertex_dict_arg for y in vertex_dict_arg[x]]
    
    return vertex_dict_arg, contracted_edge_list

# random_pair = random.choice(edge_list)
# print(random_pair)
# a,b = update_graph(random_pair[0],random_pair[1], vertex_dict_copy)
# print(b)

In [26]:
def minimum_cuts(vx_dict_arg = vertex_dict, eg_list_arg = edge_list, iterations  = 25):

    def solvemodel(x = vx_dict_arg, \
                   y = eg_list_arg):
        
        vx_dict_copy = copy.deepcopy(x)
        eg_list_copy = copy.deepcopy(y)
        
        while len(vx_dict_copy) > 2:
    
            pair = random.choice(eg_list_copy)
    
            vx_dict_copy, eg_list_copy  = \
            update_graph(pair[0], pair[1], vx_dict_copy)
        
        return vx_dict_copy, eg_list_copy
    
    # return [len(solvemodel()[1]) for i in range(0,iterations)]
    return min([len(solvemodel()[1]) for i in range(0,iterations)])

minimum_cuts()

34

# Lesson 1 Week 4

## Video 1 ~ Video 2

#### Motivations

Sorting can at most reduce the upperbound to $O(nlog(n))$. 
1. apply mergesort
2. once sorted, return $ith$ element of sorted array (this is called reduction - reduction of problems)


- Reduction to sorting, can do no better than $O(nlog(n))$.
- Selection can beat sorting
    - Selection can have linear time, $O(n)$

#### Random Selection

Pseudocode
```
RSelect(array A, length n, order statistic i)
0. 
if n == 1, return A[0]
1. 
choose pivot p from A uniformly at random
2. 
partition A around p
let j = new index of p
3. 
if j == i, return p
if j > i, return RSelect(1st part of A, j-1, i)
if j < i, return RSelect(1st part of A, n-j, i-j) 
```

- The problem: Given an array with $n$ distinct numbers, find the $i$th order statistic.
    - worst possible time (extremely unlucky): $\theta(n^2)$
    - best pivot - median: - Key: find balanced split.
        - $T(n) \leq T(n/2) + O(n)$
        - ==> $T(n) = O(n)$



*RandomSelect Theorem*: For every input array of length $n$, the average running time of RSelect is $O(n)$. 
- no assumption on data
- "average" over the algorithm

#### RandomSelection Analysis
- selection is faster than sorting!
- RSelect uses $\leq Cn$ operations outside of the recursive call
- only does one recursion in one branch

Introduce the concept of phase $j$:
- the current array size is between $(3/4)^{j+1}\times n$ and $(3/4)^{j}\times n$
- Notice that we can get stuck on phase $j$ for more than one period, when our selection does not all into the a region of size $[(3/4)^{j+1}\times n, (3/4)^{j}\times n]$.

$X_{j}$ is the # of recursive calls (or # of subproblems) during phase $j$

Expected running time of RSelect $\leq E[X_{j} \times C \times \Sigma_{phase j} (3/4)^{j}n ]$

$E[X_{j}] \leq $ expected number of times you need to flip a fair coin to get one head.
- heads == good pivot, tails == bad pivot

Let $N = $ number of coin flips until you get heads.
- This is a geometric random variable.
- $E[N] = 1 + (1/2) \times E[N]$
- $E[X_{j}] \leq E[N]$
- so that Expected running time of RSelect $\leq E[X_{j} \times C \times \Sigma_{phase j} (3/4)^{j}n ] \leq 8Cn$

## Video 3 ~ Video 4

#### Deterministic Selection

If we are not able to do random selection, what could we do?

Algorithm: median of medians.

```
Deterministic ChoosePivot(A,n)
1. logically break A into groups of 5, sort each group. (e.g. using merge sort)
2. C = the n/5 "middle elements" (copy n/5 medians into new array C)
3. p = Select(C, n/5, n/10) --- (step 1 ~ 3 == choosePivot)
4. Partition A around p
5. if j == i, return p
6. if j > i, return RSelect(1st part of A, j-1, i)
7. if j < i, return RSelect(1st part of A, n-j, i-j) 
```
2 recursive calls with linear time $O(n)$
- 1 call in line 1 to 3,
- 1 call in either 6 or 7

**DSelectTheorem**: For every input array of length $n$, DSelect runs in $O(n)$ time.
- warning: (1) worse constant compared to RSelect (2) we need additional array $C$ for copy over the arrays, but in RSelect, we do not need this additional storage.


1. Sorting an array with 5 elments takes $\leq 120$ operations, that is linear time. Because $(6 \times 5) \times(log_{2}(5)+1)$
    - so linear time for step 1
2. Step 2: (copying) also takes $\theta(n)$
3. Step 3: 1 recursion, but it is $T(n/5)$
4. Step 4: $\theta(n)$
5. for step 5,and step 6, we do not know! It depends on the length


We want to prove $T(n)\leq Cn + T(n/5) + T(?)$ ($Cn$ is outside the recursive call)
- key lemma: 2nd recursive acll (in line 6 and 7) guaranteed to be on an array of size $\leq \frac{7n}{10}$ roughly
- upshot: can replace $?$ by $7n/10$
- Proof of lemma:
    - let $k = n/5 = $ nb of groups
    - let $x_{i}= ith $  order middle elements
    - so pivot = $X_{k/2}$
    - imagine the matrix-like dotplot
        - the southwest part and the northeast part.

- Median of medians: break into groups of 5; sort each group; pick the median of median of the $n/5$ first round winners.
    - it does indeed run in linear time.
    - We are guaranteed to have a 30%-70% split.

*Now the big question*: "a sensible trade off ... Have we kept the work required to compute this 30/70 split small enough that we get the desired linear running time. Or is the cost of finding a pretty good pivot outweighing the benefit of having guaranteed good splits? "

Notes: ## In Master method, every subproblem has equal length. ## Comparion-based sorting (merge sort, quick sort) has worst case running time $\Omega(nlogn)$ 

**VIDEO 6**: confusions. recheck.

## Video 7 ~ Video 11

#### Graphs
- vertices aka nodes ($V$)
- edges($E$) = pairs of vertices
    - can be undirected (unordered pair) 
    - or directed (ordered pair) (aka arcs)
- a cut is a partition of $V$ into two non-empty sets $A$ and $B$
    - 3 possible cases for edges
- crossing-edges, only from left to right (can swap $A$ and $B$)
    - one endpoint in each $(A,B)$
    - tail in $A$, head in $B$ (directed)
- minimum cut problem
    - Input an undirected graph $G=(V,E)$ [parallel edges allowed]
    - Goal: compute a cut with fewest nb of crossing edges. (a min cut)
- A few applications
    - the most efficient way to disrupt one country's transportation network.
    - image prediction: use edge weights to identify an object

What is the input graph? We have two parameters $n,m$: number of vertices and number of edges.
- In most (but not all) applcations, $m$ is between $\Omega(n)$ and $O(n^{2})$
- sparse, more on the $\Omega(n)$ side
- dense, more on the $O(n^{2})$ side.

Adjacency Matrix
- $n \times n $ 0-1 matrix $A$

Adjacency lists ($\theta(m+n)$)
- array of vertices ($\theta(n)$)
- array of edges ($\theta(m)$)
- each edge points to its endpoints ($\theta(m)$)
- each vertex points to edges incident on it ($\theta(m)$)! reference to the 3rd case! 3rd and 4th are one-to-one correspondence.)
- So in total ($\theta(m+n)$ or $\theta(max(m,n))$)
- comparisons
    - perfect for graph search
    - for web graph, it is better than Ajacency Matrix.


#### Random contraction
While there are more than 2 vertices:
- pick a remaining edge $(u,v)$ uniformly at random
- merge (or "contract") $u$ and $v$ into a single vertex
- remove self-loops
- return graph represented by final 2 vertices.

Randomized algoritm can produce different outcomes.
- can obtain min cut
- can obtain more than min cut.

**what is the probability that we successfully obtain a particular minimum cut $(A,B)$**
- $k = $ # of edges crossing $(A,B)$.
- suppose an (min cut) edge of $F$ is contracted at some point, ==> algorithm will not output $(A,B)$
- suppose only edges inside $A$ or inside $B$ get contracted ==> algorithm will output $(A,B)$
- $Pr$[output is (A,B)] == $Pr$[never contracts an edge of $F$]
    - Let $S_{i}$ == event that an edge of $F$ contracted in iteration $i$.
    - $Pr[\lnot S_{1} \cap \lnot S_{2} ... \cap \lnot S_{n-2}]$
    - $Pr[S_{1}]$ = nb. of crossing edges / # of edges == $k/m$ (for the first iteration.)
    - However! a function of decreasing vertices is better than a function of decreasing edges. because when we contract, the edges can decrease by more than one (e.g. self loop)
    - In contrast, the number of vertices decrease steadily.
    - Key observation: Every vertext has at least $k$ edges incident on it. or degree == $k$.
        - Reason: each vertex $v$ defines a cut $({v}, V-{v})$
    - $\Sigma_{v} degree(v) = 2m$, where $degree$ is the number of edges incident on a vertex.
    - $K$ number of edges crossing our minimum cut. We have $\Sigma_{v} degree(v) \geq Kn$
    - we have $Kn\leq2m$, since $Pr[S_{1}] = k/m$, so $Pr[S_{1}] = \frac{k}{m} \leq \frac{2}{n}$
    - Now, the second iteration: We do not screw up either.
        - $Pr[\lnot S_{1} \cap \lnot S_{2}] = Pr[\lnot S_{1} | \lnot S_{2}] Pr[\lnot S_{1}]$
        - $Pr[\lnot S_{1}] \geq 1-\frac{2}{n}$
        - $Pr[\lnot S_{1} | \lnot S_{2}] = 1 - k/(nb-Remaining-Edges)$
        - but we do not know nb_remaining_edges! (we know it is at most $m-1$, but what we need is a lower bound.) we know there are $N-1$ vertices remaining. how could we relate vertices to edges? 
        - using same argument as in the first iteration, we have $nb-remaining-edges \geq \frac{1}{2} K (n-1)$ because # remaining_edges $ \times 2 \geq K (n-1)$
        - So that $Pr[\lnot S_{1}] \geq 1-\frac{2}{n-1}$
        - So that $Pr[\lnot S_{1} \cap \lnot S_{2} ... \cap \lnot S_{n-2}] \geq \frac{2}{n(n-1)}$, this is already remarkable.

Repeated trials: run the basic algorithm a large number N times, remember the smallest cut found.
- Let $T_{i} = $ event that the cut $(A,B)$ is found on the $i$th try. by definition, different  $T_{i}$'s are independent
- $Pr[all-trials-fail]$ = $Pr[\lnot T_{1} \cap \lnot T_{2} \cap ... \lnot T_{N}] = \prod_{i=1}^{N} Pr[\lnot T_{i}] \leq \frac{1}{N}$
- Running time polynomial in $n$ and $m$ but slow $\Omega(n^{2} m)$


#### *The number of minimum cuts: *
- A graph can have many distint minimum cuts.
    - for any tree with $n$ vertices, we have $n-1$ different minium cuts
- The largest number of minimum cuts (given minimum cross-edges)
    - lower bound: $C_{n}^{2}$, e.g. n-cycle
    - upper bound: 
        - imagine that we list/enumerate all the minimum cuts from $(A_{1},B_{1})$, $(A_{2},B_{2})$, ..., $(A_{t},B_{t})$
        - focus on a particular minimum cut $(A_{i},B_{i})$
        - $Pr[output = (A_{i},B_{i})] \geq \frac{2}{n(n-1)} = \frac{1}{C_{n}^{2}}$
        - let the event "output = $(A_{i}, B_{i})$" be $S_{i}$.
        - $S_{i}$'s are disjoint
            - $\frac{t}{C_{n}^{2}} \leq 1$
        
### References:
1. https://stackoverflow.com/questions/7374748/whats-the-difference-between-a-python-property-and-attribute
